# Decision Trees

Decision trees using scikit-learn

The Wisconsin Breast Cancer Dataset(WBCD) can be found here(https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data)

This dataset describes the characteristics of the cell nuclei of various patients with and without breast cancer. The task is to classify a decision tree to predict if a patient has a benign or a malignant tumour based on these features.

Attribute Information:
```
#  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)
```

The following powershell commands were run to download the dataset

```pwsh
Invoke-WebRequest -Uri https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data -OutFile breast-cancer-wisconsin.data
```

In [1]:
# Import everything
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
# Visualize data
headers = ["ID","CT","UCSize","UCShape","MA","SECSize","BN","BC","NN",
    "Mitoses","Diagnosis"]
data = pd.read_csv('breast-cancer-wisconsin.data', na_values='?',    
         header=None, index_col=['ID'], names = headers) 
data = data.reset_index(drop=True)
data = data.fillna(0)
data.describe()

,CT,UCSize,UCShape,MA,SECSize,BN,BC,NN,Mitoses,Diagnosis
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.463519,3.437768,2.866953,1.589413,2.689557
std,2.815741,3.051459,2.971913,2.855379,2.214300,3.640708,2.438364,3.053634,1.715078,0.951273
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000,4.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


## Part 1: Implementation


### Task 1: Implement a Decision Tree

1. a) Implement a decision tree (you can use decision tree implementation from existing libraries).

In [3]:
# Data for the classifier
np_data = np.array(data)
# Training and testing split
x_train, x_test, y_train, y_test = train_test_split(np_data[:,:-1], 
    np_data[:,-1], test_size=30, shuffle=True)
# Classifier using sklearn
clf = tree.DecisionTreeClassifier()

In [4]:
# Train it
clf.fit(x_train, y_train)
# Test it
y_pred = clf.predict(x_test)
print(f"Predictions: {y_pred}")
print(f"Actual: {y_test}")
print(f"Errors: {np.sum(y_pred != y_test)}")

Predictions: [2. 2. 2. 2. 4. 2. 2. 2. 4. 2. 4. 2. 2. 2. 2. 2. 2. 4. 2. 2. 4. 2. 2. 2.
 2. 2. 2. 4. 2. 4.]
Actual: [2. 2. 4. 2. 4. 2. 2. 2. 4. 2. 4. 2. 2. 2. 2. 2. 2. 4. 2. 2. 4. 2. 2. 2.
 2. 2. 2. 4. 2. 4.]
Errors: 1


In [5]:
# Save as a DOT file
dot_data = tree.export_graphviz(clf, out_file="tree1.dot", 
    feature_names=headers[1:-1], class_names=["Benign", "Malignant"],
    filled=True, rounded=True, special_characters=True)

The decision tree was converted from DOT to PNG using [onlineconvertfree](https://onlineconvertfree.com/convert-format/dot-to-png/). The tree is shown below (for a particular run)

<img alt="First decision tree" src="./figures/tree1.png" width=1000 />

This could be done using graphviz, but the current build seems to have an [issue](https://stackoverflow.com/questions/69989691/how-to-resolve-attributeerror-module-graphviz-backend-has-no-attribute-encod) on Windows platforms (as of 24 Nov 2021 at 11:30 AM)


In [13]:
# Predict a probability value (based on entropy values of node)
clf.predict_proba(x_test[[2]])

array([[0.97303922, 0.02696078]])

### Task 2: Experiment

1. b) Train a decision tree object of the above class on the WBC dataset using misclassification rate, entropy and Gini as the splitting metrics.

In [6]:
# Different metrics
clf_gini = tree.DecisionTreeClassifier(criterion='gini')
clf_ent = tree.DecisionTreeClassifier(criterion='entropy')
# Training
clf_gini.fit(x_train, y_train)
clf_ent.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy')

### Task 3: Accuracies

1. c) Report the accuracies in each of the above splitting metrics and give the best result.

Accuracy here is determined based on number of misclassifications (mismatches) on the test set. More the misclassifications, lesser will be the accuracy (as it'll generalize worse). Accuracy is therefore the percentage of test samples classified correctly.

In [7]:
# Accuracies
mis_gini = np.sum(y_test != clf_gini.predict(x_test))
mis_ent = np.sum(y_test != clf_ent.predict(x_test))
acc_gini = 1 - (mis_gini / len(y_test))
acc_ent = 1 - (mis_ent / len(y_test))
print(f"Gini: {mis_gini} wrongs, {acc_gini*100:.2f}% accurate")
print(f"\tDepth = {clf_gini.get_depth()}")
print(f"\tNumber of leaves = {clf_gini.get_n_leaves()}")
print(f"Entropy: {mis_ent} wrongs, {acc_ent*100:.2f}% accurate")
print(f"\tDepth = {clf_ent.get_depth()}")
print(f"\tNumber of leaves = {clf_ent.get_n_leaves()}")

Gini: 1 wrongs, 96.67% accurate
	Depth = 10
	Number of leaves = 37
Entropy: 2 wrongs, 93.33% accurate
	Depth = 9
	Number of leaves = 31


Gini was more accurate (with more leaves), but both were nearly the same. Different runs could also give the entropy method a higher accuracy.

In [8]:
# Result
p1t3_best_clt = clf_gini

### Task 4: Experiment more

1. d) Experiment with different approaches to decide when to terminate the tree (number of layers, purity measure, etc). Report and give explanations for all approaches. 

In [9]:
# List of all hyperparameters to test
test_hps = [    # max_depth, criterion, splitter
    [8, 'gini', 'random'],      # 0
    [5, 'gini', 'best'],        # 1
    [5, 'gini', 'random'],      # 2
    [1, 'gini', 'random'],      # 3
    [8, 'entropy', 'random'],   # 4
    [5, 'entropy', 'best'],     # 5
    [5, 'entropy', 'random'],   # 6
    [3, 'entropy', 'random'],   # 7
    [1, 'entropy', 'best'],     # 8
]
clfs = []   # Store all classifiers
acc_vals = []   # Accuracy values

In [10]:
# Generate all classifiers
for i, (max_d, c, s) in enumerate(test_hps):
    # Classifier
    clf = tree.DecisionTreeClassifier(criterion=c, splitter=s, 
        max_depth=max_d)
    # Train
    clf.fit(x_train, y_train)
    # Evaluate
    num_misc = np.sum(y_test != clf.predict(x_test))
    acc = 1 - (num_misc / len(y_test))
    print(f"Test {i} acc: {acc*100:.2f}%")
    # Store
    clfs.append(clf)
    acc_vals.append(acc)

Test 0 acc: 93.33%
Test 1 acc: 96.67%
Test 2 acc: 96.67%
Test 3 acc: 86.67%
Test 4 acc: 100.00%
Test 5 acc: 96.67%
Test 6 acc: 96.67%
Test 7 acc: 96.67%
Test 8 acc: 90.00%


Various hyper-parameters were tested in the `test_hps` list. Ultimately, the hyper-parameters that increased accuracy were kept, and those that reduced accuracy were removed. A fair mix of the gini and entropy classification criteria was done, along with even the splitting criteria. Different depths of the tree was also explored (along with the interesting case of _single_ depth)

In [11]:
# Save the tree for preview
tree.export_graphviz(clfs[4], out_file="tree2.dot", 
    feature_names=headers[1:-1], class_names=["Benign", "Malignant"],
    filled=True, rounded=True, special_characters=True)

Some interesting _single_ decision trees were also found. Note that they actually satisfy (most of) the test dataset. An example is shown below

<img src="./figures/single-dtree.png" width=300 alt="Single decision on UCSize" />

This is no better than a hard threshold on a single feature. However, a tree with more decisions (nodes) will probably generalize better since it takes more features into account. An example is the tree below

<img src="./figures/fivel-dtree.png" width=1000 alt="Decision tree with more nodes" />

It can be seen that through either case (in the top decision), there are chances of getting `Malignant` or `Benign` based on various factors. 

Note that some of the leaf nodes in the image above have entropy values that are high. You can consider them to be more-or-less inconclusive (less sure) decisions.


## Part 2: Random Forests

2. What is boosting, bagging and  stacking?
Which class does random forests belong to and why?

### Section 1: Theory

Description of the terms and the reason why random forests should be bagging.



**Ensemble Learning**

Ensemble learning is a method of using multiple weak learners to create a strong learner. Combining the predictions from multiple learners (models) yields a model that can form a stronger consensus on the prediction (majority vote, for example). Compared to single models, this type of learning can yield improved efficiency and accuracy. The two main types of ensemble learning techniques are _bagging_ and _boosting_. In bagging, weak learners are trained in parallel whereas in boosting, weak learners are trained sequentially. Usually bagging is used on weak learners that have high variance and low bias (overfitting) whereas boosting is used when there is high bias and low variance (underfitting), but this isn't a fixed rule.

**Boosting**

Boosting is a set of ensemble techniques that require training in a sequential manner. Some types are briefly described below
-  Adaptive boosting is a method of identifying misclassified data-points and adjusting their weights so that in the next iteration, they're learned correctly. This way, the algorithm gradually improves over time.
- Gradient boosting is a method that adds predictors to an ensemble with each one correcting for the errors of its predecessor. It trains on the residual errors of the previous predictor.
- Extreme gradient boosting (XGBoost) allows gradient boosting to happen efficiently, using parallelization.

**Bagging**

Bagging involves forming multiple models and having them reach a consensus. The algorithm (as introduced by [Leo Breiman](https://doi.org/10.1023/A:1018054314350)) involves the following steps
1. Bootstrapping: Creating a diverse selection of the dataset (with replacement, that is a sample can occur across various selections). These samples (smaller datasets) are called bootstrap samples.
2. Parallel training: These samples are the trained on models independently of one-another. The result is that various weak learners can be produced, who potentially learn different / diverse aspects of the data.
3. Aggregation: An average or a majority vote of these learners is taken to decide the output.

> Reference: IBM Blog on [bagging](https://www.ibm.com/cloud/learn/bagging) and [boosting](https://www.ibm.com/cloud/learn/boosting)

**Stacking**

Stacking, or stacked generalization, is an ensemble algorithm that involves combining multiple machine learning models in an efficient manner. There are base models that are individual models that train on a dataset, and then there are meta-models that learn how to generate the correct predictions given the predictions of each of these models. The goal is for the meta-model to learn the trusting weightage of the base models by understanding the areas in which the particular base models differ in their predictions.

> Reference: Blog [machinelearningmastery](https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/)

**Random Forests**

Random forests is basically an algorithm that creates a voting on decision trees. So it is more like a bagging algorithm.


### Section 2: Implementing Random Forests

3. Implement random forest algorithm using different decision trees.

> **Note**: Do not use inbuilt libraries for implementing random forests. The least expected parameters that are to be given as input are Max_depth, min_size, n_trees, n_features, criterion, and of course the data.



### Section 3: Comparison

4. Report the accuracies obtained after using the Random forest algorithm and compare it with the best accuracies obtained with the decision trees. 

## Part 3: Separate solution

5. Submit your solution as a separate pdf in the final zip file of your submission


Compute a decision tree with the goal to predict the food review based on its smell, taste and portion size.

<img src="./figures/decision_trees.png" alt="Dataset for food reviews" width=400 />

(a) Compute the entropy of each rule in the first stage.

(b) Show the final decision tree. Clearly draw it.

Submit a handwritten response. Clearly show all the steps.

